# 优缺点
根据http://scikit-learn.org/stable/modules/tree.html#tree

## Some advantages of decision trees are:
Simple to understand and to interpret. Trees can be visualised.
Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
The cost of using the tree (i.e., predicting data) is logarithmic in the number of data points used to train the tree.
Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable. See algorithms for more information.
Able to handle multi-output problems.
Uses a white box model. If a given situation is observable in a model, the explanation for the condition is easily explained by boolean logic. By contrast, in a black box model (e.g., in an artificial neural network), results may be more difficult to interpret.
Possible to validate a model using statistical tests. That makes it possible to account for the reliability of the model.
Performs well even if its assumptions are somewhat violated by the true model from which the data were generated.
## The disadvantages of decision trees include:
Decision-tree learners can create over-complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning (not currently supported), setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.
Decision trees can be unstable because small variations in the data might result in a completely different tree being generated. This problem is mitigated by using decision trees within an ensemble.
The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree. This can be mitigated by training multiple trees in an ensemble learner, where the features and samples are randomly sampled with replacement.
There are concepts that are hard to learn because decision trees do not express them easily, such as XOR, parity or multiplexer problems.
Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.

# sci-kit learn
## 代码实现

从https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/tree/tree.py
可以看到

```python
CRITERIA_CLF = {"gini": _criterion.Gini, "entropy": _criterion.Entropy}
CRITERIA_REG = {"mse": _criterion.MSE, "friedman_mse": _criterion.FriedmanMSE}
```

这两行对应的分别是分类树和回归树进行split时的criteria。所以sklearn中支持的分类的criteria是gini和entropy，支持的回归的criteria则是mse，当然还有一个friedman提出来的mse（有啥区别？待查）

分类树的生成是比较简单的：对于当前的分裂节点，遍历所有的feature，看哪个feature分裂的结果能够得到最大的gini系数或者entropy。Gini和Entropy都是用来描述变量的不确定性，它们的值越大，则表明变量的不确定性越大，也就是说变量取得各种值的概率相近。

回归树的生成还是有那么一点小麻烦的，是采用的启发式的方法。

在sklearn中的tree的split有两种，dense和sparse，这个指的是输入的矩阵是sparse的还是dense的，跟算法本身没有关系。


sklearn中的ExtraTreeRegressor继承了DecisionTreeRegressor.

```python
    class sklearn.tree.DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, presort=False)
    
```

```python
class sklearn.tree.ExtraTreeRegressor(criterion='mse', splitter='random', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', random_state=None, max_leaf_nodes=None)
```

标准的DecisionTreeRegressor的splitter是best策略，ExtraDecisionTree继承了标准的DecisionTreeRegressor，主要的改变就是参数传递是将splitter设置成了random。ExtraTreeRegressor中的Extra其实是extremely random的简称。什么意思呢？对于标准的regression tree，在每一个节点进行分裂的时候，与分类树类似，会遍历每一个feature，然后对于每一个feature，遍历其所取的所有的值，然后计算mse，取mse最小的作为最好的分割。

但是ExtraTreeRegressor则不同，同样的，它会遍历所有的feature，但是对于每一个feature，并不会遍历所有的取值，而是随机选择一个值作为分割值。相关代码为（其中，rand_uniform就是从当前feature的取值范围随机选择一个值）：

```python
                    current.threshold = rand_uniform(min_feature_value,
                                                     max_feature_value,
                                                     random_state)

                    if current.threshold == max_feature_value:
                        current.threshold = min_feature_value

                    # Partition
                    partition_end = end
                    p = start
                    while p < partition_end:
                        current_feature_value = Xf[p]
                        if current_feature_value <= current.threshold:
                            p += 1
                        else:
                            partition_end -= 1

                            Xf[p] = Xf[partition_end]
                            Xf[partition_end] = current_feature_value

                            tmp = samples[partition_end]
                            samples[partition_end] = samples[p]
                            samples[p] = tmp

                    current.pos = partition_end

```

sklearn对ExtraTreeRegressor的解释如下：Extra-trees differ from classic decision trees in the way they are built. When looking for the best split to separate the samples of a node into two groups, random splits are drawn for each of the max_features randomly selected features and the best split among those is chosen. When max_features is set 1, this amounts to building a totally random decision tree.

其中，特别强调：

Warning: Extra-trees should only be used within ensemble methods.

这是因为，如果只有一棵树，采用这种随机的方法生成的回归树显然mse会是很大的，离最优的mse可能会有相当的距离。但是如果是ensemble一堆的树，则分割值本身的随机性会解决这个问题。